In [1]:
!pip install peft
!pip install datasets
!pip install bitsandbytes
!pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.5 MB/s eta 0:00:0000:0100:01


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from datasets import load_dataset
from huggingface_hub import notebook_login
import torch

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [4]:
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
dataset_name = "multi_nli"

In [5]:
notebook_login()
#

In [6]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.float16,
    bnb_4bit_use_double_quant= False,
)

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config= bnb_config,
    device_map="auto",
)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [9]:
model = prepare_model_for_kbit_training(model)

In [10]:
import torch.nn as nn
linear_layers = []
for name, module in model.named_modules():
    if isinstance(module, nn.Linear):
        linear_layers.append(name)

print(linear_layers)

['model.layers.0.self_attn.q_proj', 'model.layers.0.self_attn.k_proj', 'model.layers.0.self_attn.v_proj', 'model.layers.0.self_attn.o_proj', 'model.layers.0.mlp.gate_proj', 'model.layers.0.mlp.up_proj', 'model.layers.0.mlp.down_proj', 'model.layers.1.self_attn.q_proj', 'model.layers.1.self_attn.k_proj', 'model.layers.1.self_attn.v_proj', 'model.layers.1.self_attn.o_proj', 'model.layers.1.mlp.gate_proj', 'model.layers.1.mlp.up_proj', 'model.layers.1.mlp.down_proj', 'model.layers.2.self_attn.q_proj', 'model.layers.2.self_attn.k_proj', 'model.layers.2.self_attn.v_proj', 'model.layers.2.self_attn.o_proj', 'model.layers.2.mlp.gate_proj', 'model.layers.2.mlp.up_proj', 'model.layers.2.mlp.down_proj', 'model.layers.3.self_attn.q_proj', 'model.layers.3.self_attn.k_proj', 'model.layers.3.self_attn.v_proj', 'model.layers.3.self_attn.o_proj', 'model.layers.3.mlp.gate_proj', 'model.layers.3.mlp.up_proj', 'model.layers.3.mlp.down_proj', 'model.layers.4.self_attn.q_proj', 'model.layers.4.self_attn.k_

In [11]:
peft_config = LoraConfig(
    lora_alpha= 16,
    lora_dropout= 0.05,
    r= 16,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules= ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"]
#     target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj", "up_proj"]
)

In [12]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 41,943,040 || all params: 8,072,204,288 || trainable%: 0.5196


In [13]:
dataset = load_dataset(dataset_name)

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

In [15]:
import random
def sample_5_percent(dataset):
    total_size = len(dataset)
    sample_size = max(1, 5 * total_size // 100)
    indices = random.sample(range(total_size), sample_size)
    return dataset.select(indices)

In [16]:
sampled_train_dataset = sample_5_percent(dataset['train'])
sampled_validation_dataset = sample_5_percent(dataset['validation_matched'])
sampled_train_dataset.shape, sampled_validation_dataset.shape

((19635, 10), (490, 10))

## Creating chat template for traingin efficiency

In [17]:
mapper = {0: 'entailment', 1: 'neutral', 2: 'contradiction'}
INSTRUCTION = "Decide whether the following statements are entailment, contradiction, or neutral."

In [18]:
def create_prompt(example, instruction, mapper):
    premise = example['premise']
    hypothesis = example['hypothesis']
    label = example['label']
    user_input = f"\nPremise: {premise}\nHypothesis: {hypothesis}\n"
    prompt = f"""<|start_header_id|>system<|end_header_id|> {instruction}<|eot_id|><|start_header_id|>user<|end_header_id|> Are these two statements entailment, contradiction, or neutral?: {user_input}<|eot_id|><|start_header_id|>assistant<|end_header_id|> {mapper[label]}<|eot_id|>"""
    return prompt


In [19]:
train_dataset = sampled_train_dataset.map(lambda example: {'prompt': create_prompt(example, INSTRUCTION, mapper)})

Map:   0%|          | 0/19635 [00:00<?, ? examples/s]

In [20]:
train_dataset

Dataset({
    features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label', 'prompt'],
    num_rows: 19635
})

In [21]:
def preprocess_function(examples):
    outputs = tokenizer(examples['prompt'], truncation=True, max_length=None)
    return outputs

In [22]:
train_tokenized_dataset = train_dataset.map(preprocess_function, batched=True, remove_columns=["promptID", "pairID", "premise", "premise_binary_parse", "premise_parse", "hypothesis", "hypothesis_binary_parse", "hypothesis_parse", "genre", "prompt", "label"])

Map:   0%|          | 0/19635 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [23]:
train_tokenized_dataset

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 19635
})

In [24]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
trainer = Trainer(
    model=model,
    train_dataset=train_tokenized_dataset,
    args=TrainingArguments(
        # num_train_epochs= 1,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        logging_steps= 30,
        warmup_steps=5,
        max_steps=600,
        learning_rate=2e-5,
        weight_decay= 0.01,
        fp16= False,
        bf16= False,
        max_grad_norm= 0.3,
        group_by_length= True,
        optim="paged_adamw_8bit",
        output_dir= "/content/drive/MyDrive/weigths",
        report_to="none",
    ),
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

2024-06-01 15:49:59.479544: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-01 15:49:59.479681: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-01 15:49:59.565382: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_con

In [28]:
model.config.pad_token_id=tokenizer.pad_token_id
model.config.pad_token_id

128009

In [29]:
model.train()
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
30,3.063100
60,1.529800
90,1.512400
120,1.509300
150,1.282200
180,1.632700
210,1.351300
240,1.372100
270,1.346000
300,1.087200


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=600, training_loss=1.41478209177653, metrics={'train_runtime': 7807.1143, 'train_samples_per_second': 1.23, 'train_steps_per_second': 0.077, 'total_flos': 3.722169493005926e+16, 'train_loss': 1.41478209177653, 'epoch': 0.49})

In [30]:
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): Linear

In [95]:
def create_prompt2(example, instruction):
    premise = example['premise']
    hypothesis = example['hypothesis']
    user_input = f"\nPremise: {premise}\nHypothesis: {hypothesis}\n"
    prompt = f"<|start_header_id|>system<|end_header_id|> {instruction}<|eot_id|><|start_header_id|>user<|end_header_id|> Are these two statements entailment, contradiction, or neutral?: {user_input}<|eot_id|>"
    return prompt


In [115]:
sampled_validation_dataset = sampled_validation_dataset.map(lambda example: {'prompt': create_prompt2(example, INSTRUCTION)})

Map:   0%|          | 0/490 [00:00<?, ? examples/s]

In [116]:
sampled_validation_dataset

Dataset({
    features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label', 'prompt'],
    num_rows: 490
})

In [105]:
def preprocess_function2(examples):
    outputs = tokenizer(examples['prompt'], truncation=True, max_length=512, padding=True, return_tensors="pt")
    return outputs

In [140]:
val_match = sampled_validation_dataset.map(preprocess_function2, batched=True, remove_columns=["promptID", "pairID", "premise", "premise_binary_parse", "premise_parse", "hypothesis", "hypothesis_binary_parse", "hypothesis_parse", "genre", "prompt"])

Map:   0%|          | 0/490 [00:00<?, ? examples/s]

In [141]:
val_match

Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 490
})

In [135]:
def zero_shot_prompting(input_ids, attention_mask, model, max_length=10, temperature=0.2, top_p=0.9, top_k=50):
    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=input_ids.shape[1] + max_length
            top_p=top_p,
            top_k=top_k,
            temperature=temperature,
            pad_token_id=tokenizer.pad_token_id,
        )
    generated_tokens = outputs[0][inputs['input_ids'].shape[1]:]
    generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    return generated_text

In [136]:
predictions = []
for sample in val_match:
    input_ids = torch.tensor(sample['input_ids']).unsqueeze(0).to(device)
    attention_mask= torch.tensor(sample['attention_mask']).unsqueeze(0).to(device)
    predictions.append(zero_shot_prompting(input_ids, attention_mask, model))

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='le

In [142]:
numerical_predictions = []
for prediction in predictions:
    pred_label = -1
    if "entailment" in prediction.lower():
        pred_label = 0
    elif "neutral" in prediction.lower():
        pred_label = 1
    elif "contradiction" in prediction.lower():
        pred_label = 2
    numerical_predictions.append(pred_label)

In [144]:
from sklearn.metrics import accuracy_score
print(accuracy_score(val_match['label'], numerical_predictions))

0.36122448979591837
